<a href="https://colab.research.google.com/github/jodog0412/DACON/blob/main/TRAVEL_BOOL_ANALYSIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab/TRAVEL_PRODUCT
!ls

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
train = pd.read_csv('dataset/train.csv')
test = pd.read_csv('dataset/test.csv')
sample_submission = pd.read_csv('dataset/sample_submission.csv')
train.info()
train.isna().sum()

In [ ]:
# 결측치 처리
def handle_na(data):
    temp = data.copy()
    for col, dtype in temp.dtypes.items():
        if dtype == 'object':
            value = 'unknown'
        elif dtype == int or dtype == float:
            value = 0
        temp.loc[:,col] = temp[col].fillna(value)
    return temp
train_nona=handle_na(train)
test_nona = handle_na(test)
train_nona.info()
test_nona.info()

In [ ]:
object_columns = train_nona.columns[train_nona.dtypes == 'object']

from sklearn.preprocessing import LabelEncoder
train_enc = train_nona.copy()
test_enc=test_nona.copy()
for o_col in object_columns:
    encoder = LabelEncoder()
    encoder.fit(train_enc[o_col])
    train_enc[o_col] = encoder.transform(train_enc[o_col])
    test_enc[o_col] = encoder.transform(test_enc[o_col])

train_enc.info()
train_enc.describe(include="number")
# train_enc["MonthlyIncome"].hist(bins=100)

In [ ]:
from sklearn import svm
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC

x_train = train_enc.drop(columns=['id','ProdTaken'])
x_test = test_enc.drop(columns=['id'])
y_train = train_enc[['ProdTaken']]

clf = make_pipeline(RobustScaler(), SVC(gamma='auto'))
clf.fit(x_train, y_train)
prediction=clf.predict(x_test)
cm = confusion_matrix(y_train, prediction[:1955]>0.5)
cm
# sample_submission['ProdTaken'] = prediction
# sample_submission.to_csv('submission.csv',index = False)
# import numpy as np
# import torch
# from torch import nn
# from torch import optim
# import matplotlib.pyplot as plt
# torch.manual_seed(1)

# X=torch.tensor(x_train.values)
# Y=torch.tensor(y_train.values)
# model = nn.Sequential(
#     nn.Linear(18, 36),
#     nn.LeakyReLU(),
#     nn.Linear(36, 18),
#     nn.LeakyReLU(),
#     nn.Linear(18, 9),
#     nn.LeakyReLU(),
#     nn.Linear(9, 4), 
#     nn.LeakyReLU(),
#     nn.Linear(4, 1) 
# )
# loss_fn = nn.functional.binary_cross_entropy_with_logits
# optimizer = optim.Adam(model.parameters(), lr=0.01)
# losses=[]
# epochs=25000
# for epoch in range(epochs+1):
#     optimizer.zero_grad() 
#     y_pred = model(X.float())  
#     loss = loss_fn(y_pred, Y.float())
#     loss.backward()  
#     optimizer.step()  
#     losses.append(loss.item())
# plt.plot(losses)

In [ ]:
# T=torch.tensor(x_test.values)
# model.eval()
# prediction=model(T.float())>= torch.Tensor([0.5])
# prediction=prediction.int()
# print(prediction[:10])

In [ ]:
sample_submission['ProdTaken'] = prediction.detach().numpy()
sample_submission.head()
sample_submission.to_csv('submission.csv',index = False)